### Imports

In [25]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 2020 Census Population data

In [26]:
url = 'https://www.census.gov/quickfacts/quickfacts/geo/dashboard/US/POP010220'
res = requests.get(url)

In [27]:
soup = BeautifulSoup(res.content)

In [28]:
table = soup.find(attrs={'class':'qf-graph-scroll'})
# soup.find_all(attrs={'class':'qf-positive'})
tbody = table.find_all(attrs={'class':"qf-graph-geo"})
tr = tbody[1]
state = tr.find('a').attrs['data-title']
tr.find(attrs={'class':'qf-positive'}).attrs['data-value']

'29145505'

In [29]:
pop_2020 = []
for tr in tbody:
    state = tr.find('a').attrs['data-title']
    pop = tr.find(attrs={'class':'qf-positive'}).attrs['data-value']
    
    state_pop = {'State': state, '2020 Population': pop}
    pop_2020.append(state_pop)

In [30]:
pop_2020 = pd.DataFrame(pop_2020)

### 2010 Census Population Data

In [31]:
url2 = 'https://www.census.gov/quickfacts/quickfacts/geo/dashboard/US/POP010210'
res2 = requests.get(url2)

In [32]:
soup2 = BeautifulSoup(res2.content)
table2 = soup2.find(attrs={'class':'qf-graph-scroll'})
tbody2 = table2.find_all(attrs={'class':"qf-graph-geo"})

In [33]:
pop_2010 = []
for tr in tbody2:
    state = tr.find('a').attrs['data-title']
    pop = tr.find(attrs={'class':'qf-positive'}).attrs['data-value']
    
    state_pop = {'State': state, '2010 Population': pop}
    pop_2010.append(state_pop)

pop_2010 = pd.DataFrame(pop_2010)

In [34]:
population = pd.merge(pop_2010, pop_2020)
population.head()

,State,2010 Population,2020 Population
0,California,37253956,39538223
1,Texas,25145561,29145505
2,New York,19378102,20201249
3,Florida,18801310,21538187
4,Illinois,12830632,12812508


In [35]:
population.to_csv('Data/Population_data_2010_&_2020.csv')

### Excess Deaths Data

In [36]:
deaths = pd.read_csv('Ignore/Excess_Deaths_Associated_with_COVID-19.csv')
deaths.head(100)

,Week Ending Date,State,Observed Number,Upper Bound Threshold,Exceeds Threshold,Average Expected Count,Excess Estimate,Total Excess Estimate,Percent Excess Estimate,Year,Type,Outcome,Suppress,Note
0,2017-01-07,Alabama,1121.0,1136,False,1059,62,29601,5.852700,2017,Predicted (weighted),All causes,NaN,NaN
1,2017-01-14,Alabama,1130.0,1140,False,1067,63,29601,5.906102,2017,Predicted (weighted),All causes,NaN,NaN
2,2017-01-21,Alabama,1048.0,1142,False,1071,0,29601,0.000000,2017,Predicted (weighted),All causes,NaN,NaN
3,2017-01-28,Alabama,1026.0,1142,False,1070,0,29601,0.000000,2017,Predicted (weighted),All causes,NaN,NaN
4,2017-02-04,Alabama,1036.0,1142,False,1068,0,29601,0.000000,2017,Predicted (weighted),All causes,NaN,NaN
5,2017-02-11,Alabama,1058.0,1136,False,1062,0,29601,0.000000,2017,Predicted (weighted),All causes,NaN,NaN
6,2017-02-18,Alabama,1060.0,1132,False,1057,3,29601,0.283804,2017,Predicted (weighted),All causes,NaN,NaN
7,2017-02-25,Alabama,1099.0,1126,False,1051,48,29601,4.567542,2017,Predicted (weighted),All causes,NaN,NaN
8,2017-03-04,Alabama,1081.0,1119,False,1042,39,29601,3.743414,2017,Predicted (weighted),All causes,NaN,NaN
9,2017-03-11,Alabama,1011.0,1113,False,1036,0,29601,0.000000,2017,Predicted (weighted),All causes,NaN,NaN


In [105]:
covid = deaths[(deaths['Type']=='Predicted (weighted)')&((deaths['Outcome'] == 'All causes') | (deaths['Outcome'] == 'All causes, excluding COVID-19')) & 
       (deaths['Year'] == 2020) | (deaths['Year'] == 2021) | (deaths['Year'] == 2022)]

In [106]:
covid['Type'].value_counts()

Predicted (weighted)    16956
Unweighted               5670
Name: Type, dtype: int64

In [107]:
covid['Outcome'].value_counts()

All causes                        14148
All causes, excluding COVID-19     8478
Name: Outcome, dtype: int64

In [169]:
covid = deaths[(deaths['Type'] == 'Predicted (weighted)') & 
               ((deaths['Outcome'] == 'All causes') | (deaths['Outcome'] == 'All causes, excluding COVID-19')) & 
               (deaths['Year'].isin([2020, 2021, 2022]))]

def covid_deaths(data):
    grouped = data.groupby(['Year', 'State', 'Outcome'])['Excess Estimate'].sum().reset_index()
    pivoted = grouped.pivot(index=['Year', 'State'], columns='Outcome', 
                            values='Excess Estimate').reset_index()
    pivoted = pivoted.fillna(0)
    pivoted['covid'] = pivoted['All causes'] - pivoted['All causes, excluding COVID-19']
    pivoted.rename(columns={'Year': 'Location'}, inplace=True)
    results = pivoted.to_dict(orient='records')

    return results


In [170]:
deaths_covid = pd.DataFrame(covid_deaths(covid))

In [171]:
deaths_covid

,Location,State,All causes,"All causes, excluding COVID-19",covid
0,2020,Alabama,9021,2684,6337
1,2020,Alaska,545,332,213
2,2020,Arizona,13186,4583,8603
3,2020,Arkansas,4992,1301,3691
4,2020,California,41279,11317,29962
5,2020,Colorado,6426,1692,4734
6,2020,Connecticut,6015,549,5466
7,2020,Delaware,1338,414,924
8,2020,District of Columbia,1171,322,849
9,2020,Florida,26556,5979,20577


In [172]:
deaths_covid = deaths_covid.pivot(index='State', columns='Location', 
                                  values=['All causes', 'covid'])

deaths_covid.columns = ['_'.join(map(str, col)) for col in deaths_covid.columns]
deaths_covid = deaths_covid.reset_index()
deaths_covid.rename(columns={'State': 'Location'}, inplace=True)
deaths_covid.rename(columns={
    'All causes_2020': 'all_causes_2020',
    'All causes_2021': 'all_causes_2021',
    'All causes_2022': 'all_causes_2022',
    'covid_2020': 'covid_2020',
    'covid_2021': 'covid_2021',
    'covid_2022': 'covid_2022',
}, inplace=True)
deaths_covid.head()

,Location,all_causes_2020,all_causes_2021,all_causes_2022,covid_2020,covid_2021,covid_2022
0,Alabama,9021,13018,6246,6337,9771,3933
1,Alaska,545,1429,686,213,804,275
2,Arizona,13186,17961,8835,8603,13536,5849
3,Arkansas,4992,6908,3854,3691,5333,2593
4,California,41279,60680,36786,29962,48834,21158


In [174]:
(deaths_covid[['covid_2020', 'covid_2021', 'covid_2022']].sum()).sum()

2045325

In [175]:
deaths_covid.to_csv('Data/Cleaned/cleaned_covid_death_state.csv', index = False)

In [43]:
# getting sum of each states excess deaths 
def excess_deaths(deaths, start_year=2017, end_year=2022):
    state_totals = {state: [0] * (end_year - start_year + 1) for state in 
                    deaths['State'].unique()}
    
    for index, row in deaths.iterrows():
        state = row['State']
        year = row['Year']
        excess_estimate = row['Excess Estimate']
        
        if start_year <= year <= end_year:
            state_totals[state][year - start_year] += excess_estimate

    result = pd.DataFrame(state_totals).T.reset_index()
    result.columns = ['Location'] + [f'Exc_deaths_{year}' for year in 
                                  range(start_year, end_year + 1)]
    
    return result


excess_deaths = excess_deaths(deaths)
excess_deaths.head()


,Location,Exc_deaths_2017,Exc_deaths_2018,Exc_deaths_2019,Exc_deaths_2020,Exc_deaths_2021,Exc_deaths_2022
0,Alabama,2649,4062,945,20726,29283,14805
1,Alaska,417,492,639,1422,3483,1783
2,Arizona,3522,4281,1785,30955,40347,20653
3,Arkansas,3054,2199,1434,11285,15391,8969
4,California,17241,13434,1029,93875,133206,89195


In [44]:
# Getting count for each year and each state where they exceed their threshold
def count_exceeds_threshold(deaths, start_year=2017, end_year=2022):
    result = pd.DataFrame()
    
    for year in range(start_year, end_year + 1):
        
        year_data = deaths[deaths['Year'] == year]
        year_counts = year_data.groupby('State')['Exceeds Threshold'].sum().reset_index()
        year_counts = year_counts.rename(columns={'Exceeds Threshold': f'Exc_count_{year}'})
        year_counts.set_index('State', inplace=True)
        
        if result.empty:
            result = year_counts
        else:
            result = result.merge(year_counts, left_index=True, right_index=True, how='outer')
    
    result = result.reset_index().fillna(0)
    
    return result

In [45]:
def count_exceeds_threshold(deaths, start_year=2017, end_year=2022):
    results = []
    
    for year in range(start_year, end_year + 1):
        year_counts = deaths[deaths['Year'] == year]
        state_counts = {}
        
        for state in year_counts['State'].unique():
            count = year_counts[(year_counts['State'] == state) & (year_counts['Exceeds Threshold'] == 'True')].shape[0]
            state_counts[state] = count
        
        results.append({'Year': year, **state_counts})
    
    return pd.DataFrame(results)

exceeds_threshold = count_exceeds_threshold(deaths, start_year=2017, end_year=2022)
exceeds_threshold

,Year,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,New York City,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Puerto Rico,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,United States,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2019,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2021,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
deaths["Week Ending Date"] = pd.to_datetime(deaths["Week Ending Date"])
deaths["Year"] = deaths["Week Ending Date"].dt.year
exceeds_threshold = deaths[deaths["Exceeds Threshold"] == True]
result = exceeds_threshold.groupby(["State", "Year"]).size().reset_index(name="Exceeds Threshold Count")

result

,State,Year,Exceeds Threshold Count
0,Alabama,2017,3
1,Alabama,2018,15
2,Alabama,2020,99
3,Alabama,2021,106
4,Alabama,2022,87
5,Alabama,2023,21
6,Alaska,2017,6
7,Alaska,2018,6
8,Alaska,2019,3
9,Alaska,2020,34


In [48]:
# Merging the data
#finaldeaths = pd.merge(excess_deaths, exceeds_threshold, how='left')
#finaldeaths.rename(columns={'State':'Location'}, inplace=True)
#finaldeaths.head()

In [49]:
#finaldeaths.to_csv('Data/Excess_deaths&Exceeds_Threshold_data.csv')

### Population Density Data

In [50]:
url3 = 'https://wisevoter.com/state-rankings/population-density-by-state/'
res3 = requests.get(url3)
soup3 = BeautifulSoup(res3.content)

In [51]:
table3 = soup3.find('table', attrs={'id': 'shdb-on-page-table'})
tbody = table3.find('tbody')
trs = tbody.find_all('tr')

In [52]:
pop_density = []
for tr in trs:
    state = tr.find(attrs={'class':'shdb-on-page-table-body-Geo'}).text
    density = tr.find(attrs={'class':'shdb-on-page-table-body-Data'}).text.split()[0]
    
    state_density = {'State':state, 'Population Density per mi²': density}
    pop_density.append(state_density)
Pop_density = pd.DataFrame(pop_density)
Pop_density.head()

,State,Population Density per mi²
0,New Jersey,"1,283.4"
1,Rhode Island,"1,074.3"
2,Massachusetts,919.82
3,Connecticut,746.7
4,Maryland,648.84


In [53]:
Pop_density.to_csv('Data/Population_Density_data.csv')

### Mask county data

In [54]:
county_mask = pd.read_csv('Ignore/U.S._State_and_Territorial_Public_Mask_Mandates_From_April_10__2020_through_August_15__2021_by_County_by_Day.csv')
county_mask

In [55]:
# Put my code into Chatgpt who helped simplify to speed up run time
def mandate_length(data):
    county_counts = {}
    for index, row in data.iterrows():
        county_name = row['County_Name']
        mask_required = row['Face_Masks_Required_in_Public']
        state = row['State_Tribe_Territory']
        
        if mask_required == 'Yes':
            if state in county_counts:
                if county_name in county_counts[state]:
                    county_counts[state][county_name]['Count'] += 1
                else:
                    county_counts[state][county_name] = {'State': state, 'Count': 1}
            else:
                county_counts[state] = {county_name: {'State': state, 'Count': 1}}

    county_counts_list = [{'State': state, 'County': county_name, 'Count': county_counts[state][county_name]['Count']} for state in county_counts for county_name in county_counts[state]]

    return pd.DataFrame(county_counts_list)

county_counts_df = mandate_length(county_mask)

In [56]:
county_counts_df.head()

,State,County,Count
0,AL,Autauga County,267
1,AL,Baldwin County,267
2,AL,Barbour County,267
3,AL,Bibb County,267
4,AL,Blount County,267


In [57]:
county_counts_df.shape

(2423, 3)

In [58]:
county_counts_df['State'].value_counts()

TX    254
VA    133
KY    120
KS    105
IL    102
NC    100
IA     99
IN     92
OH     88
MN     87
MI     83
MS     82
PR     78
AR     75
WI     72
PA     67
AL     67
LA     64
CO     64
NY     62
CA     58
MT     56
WV     55
ND     53
WA     39
OR     36
NM     33
UT     29
MD     24
WY     23
NJ     21
NV     17
ME     16
MA     14
VT     14
NH     10
CT      8
RI      5
HI      5
AS      5
DE      3
VI      3
GU      1
DC      1
Name: State, dtype: int64

In [59]:
county_counts_df.to_csv('Data/county_mask_mandata.csv', index=False)

### Merging the data

In [60]:
def merge_dataframes():
    employ = pd.read_csv('Data/Cleaned/cleaned_employment.csv')
    income = pd.read_csv('Data/Cleaned/cleaned_income.csv')
    life = pd.read_csv('Data/Cleaned/cleaned_Life_Expentency.csv')
    insur = pd.read_csv('Data/Cleaned/cleaned_insur.csv')
    pop_dense = pd.read_csv('Data/Cleaned/cleaned_pop_dense.csv')
    pop_size = pd.read_csv('Data/Cleaned/cleaned_pop_size.csv')
    pre_cond = pd.read_csv('Data/Cleaned/cleaned_pre_condtions.csv')
    total_phys = pd.read_csv('Data/Cleaned/cleaned_total_physician.csv')
    
    merged_df = employ.merge(income, on='Location').merge(life, on='Location').merge(insur, on='Location') \
        .merge(pop_dense, on='Location').merge(pop_size, on='Location').merge(pre_cond, on='Location') \
        .merge(total_phys, on='Location').merge(excess_deaths, on='Location')

    return merged_df
merged_data = merge_dataframes()

In [61]:
merged_data.drop(columns=['Unnamed: 0'], inplace=True)
merged_data.head()


,Location,Employment_2020,Employment_2021,Employment_2022,Inc_Per_Cap_2020,Inc_Per_Cap_2021,Inc_Per_CAp_2022,Life_Exp_2020,Life_Exp_2019,Life_Exp_2018,Employer_2019,Non-Group_2019,Medicaid_2019,Medicare_2019,Military_2019,Uninsured_2019,Employer_2021,Non-Group_2021,Medicaid_2021,Medicare_2021,Military_2021,Uninsured_2021,Population Density per mi²,2010 Population,2020 Population,flu_vaccination_rate_2019,asthma_prevalence,cardiac_mortality_rate,high_bp_prevalence,copd_prevalence,kidney_disease_prevalence,diabetes_prevalence,Physicians,Physicians Rate,Active MO,Active MO Rate,Active DO,Active DO Rate,Exc_deaths_2017,Exc_deaths_2018,Exc_deaths_2019,Exc_deaths_2020,Exc_deaths_2021,Exc_deaths_2022
0,Alabama,2671005,2769464,2869931,45887,50059,50916,73.2,75.2,75.1,0.472,0.055,0.195,0.160,0.021,0.097,0.466,0.060,0.192,0.162,0.021,0.100,100.65,4779736,5024279,39.5,9.4,289.3,39.4,9.3,3.1,12.2,"10,983",224.0,"10,389",211.9,594,12.1,2649,4062,945,20726,29283,14805
1,Alaska,430840,443047,457687,61898,65662,68635,76.6,77.7,78.0,0.484,0.035,0.213,0.100,0.053,0.115,0.433,0.041,0.257,0.107,0.053,0.108,1.3,710231,733391,37.0,9.7,178.5,32.8,4.6,1.8,7.1,"2,101",287.2,"1,825",249.5,276,37.7,417,492,639,1422,3483,1783
2,Arizona,3920033,4086802,4287595,52133,56420,58442,76.3,78.8,78.7,0.451,0.052,0.210,0.161,0.015,0.111,0.450,0.054,0.213,0.162,0.015,0.106,64.96,6392017,7151502,37.1,9.8,181.8,29.9,6.0,3.6,9.8,"18,343",252.0,"15,989",219.7,"2,353",32.3,3522,4281,1785,30955,40347,20653
3,Arkansas,1639829,1686444,1755536,47147,51636,52618,73.8,75.7,75.6,0.420,0.054,0.262,0.159,0.014,0.091,0.411,0.056,0.270,0.156,0.015,0.092,58.43,2915918,3011524,40.1,9.3,284.3,38.2,9.7,3.7,12.2,"6,500",215.4,"6,132",203.2,368,12.2,3054,2199,1434,11285,15391,8969
4,California,23154091,23934549,25300974,70061,76991,77036,79.0,80.9,80.8,0.480,0.066,0.253,0.114,0.009,0.078,0.470,0.070,0.265,0.117,0.008,0.070,258.21,37253956,39538223,40.7,7.8,192.5,26.6,4.2,2.8,9.4,"113,718",287.8,"107,383",271.8,"6,333",16.0,17241,13434,1029,93875,133206,89195


In [62]:
vax_state = pd.read_csv('Data/Cleaned/vax_state.csv')
vax_state.head()

,Location,Dist_Per_100K_2021,Distributed_Per_100k_65Plus_2021,Admin_Per_100K_2021,Admin_Per_100k_65Plus_2021,Administered_Dose1_Pop_Pct_2021,Administered_Dose1_Recip_65PlusPop_Pct_2021,Series_Complete_Pop_Pct_2021,Series_Complete_65PlusPop_Pct_2021,Additional_Doses_Vax_Pct_2021,Additional_Doses_50Plus_Vax_Pct_2021,Additional_Doses_65Plus_Vax_Pct_2021,Dist_Per_100K_2022,Distributed_Per_100k_65Plus_2022,Admin_Per_100K_2022,Admin_Per_100k_65Plus_2022,Administered_Dose1_Pop_Pct_2022,Administered_Dose1_Recip_65PlusPop_Pct_2022,Series_Complete_Pop_Pct_2022,Series_Complete_65PlusPop_Pct_2022,Additional_Doses_Vax_Pct_2022,Additional_Doses_50Plus_Vax_Pct_2022,Additional_Doses_65Plus_Vax_Pct_2022,Second_Booster_50Plus_Vax_Pct,Second_Booster_65Plus_Vax_Pct,Bivalent_Booster_65Plus_Pop_Pct
0,KS,178349,1092740,140263,241990,69.4,95.0,57.1,88.2,33.8,48.5,58.3,273256,1674240,181329,327654,75.9,95.0,65.0,95.0,48.7,64.1,72.8,51.2,59.1,39.7
1,AL,164304,947963,115022,205476,58.6,92.8,47.7,79.7,27.4,38.9,49.0,242643,1399940,141328,259917,64.8,95.0,53.0,85.0,39.5,53.1,62.7,37.9,43.5,20.2
2,NJ,210259,1265690,168844,238218,83.9,95.0,70.6,89.9,34.2,48.6,58.1,317756,1912780,219584,324072,94.3,95.0,78.8,95.0,51.6,64.9,72.2,44.9,53.7,35.1
3,GA,172562,1207790,125950,217166,61.4,92.3,51.1,80.8,28.2,40.1,48.6,258794,1811340,158505,284715,68.1,95.0,57.1,86.4,43.7,58.3,67.1,42.1,48.7,26.6
4,PR,201561,947131,185830,223651,89.1,95.0,77.1,86.1,28.9,41.6,46.2,268619,1262240,248533,299619,95.0,95.0,86.7,92.8,63.4,73.8,75.6,33.1,38.6,12.3


In [63]:
state_name = {
    'AL':'Alabama',
    'AK':'Alaska',
    'AZ':'Arizona',
    'AR':'Arkansas',
    'CA':'California',
    'CO':'Colorado',
    'CT':'Connecticut',
    'DE':'Delaware',
    'DC':'District of Columbia',
    'FL':'Florida',
    'GA':'Georgia',
    'HI':'Hawaii',
    'ID':'Idaho',
    'IL':'Illinois',
    'IN':'Indiana',
    'IA':'Iowa',
    'KS':'Kansas',
    'KY':'Kentucky',
    'LA':'Louisiana',
    'ME':'Maine',
    'MD':'Maryland',
    'MA':'Massachusetts',
    'MI':'Michigan',
    'MN':'Minnesota',
    'MS':'Mississippi',
    'MO':'Missouri',
    'MT':'Montana',
    'NE':'Nebraska',
    'NV':'Nevada',
    'NH':'New Hampshire',
    'NJ':'New Jersey',
    'NM':'New Mexico',
    'NY':'New York',
    'NC':'North Carolina',
    'ND':'North Dakota',
    'OH':'Ohio',
    'OK':'Oklahoma',
    'OR':'Oregon',
    'PA':'Pennsylvania',
    'RI':'Rhode Island',
    'SC':'South Carolina',
    'SD':'South Dakota',
    'TN':'Tennessee',
    'TX':'Texas',
    'UT':'Utah',
    'VT':'Vermont',
    'VA':'Virginia',
    'WA':'Washington',
    'WV':'West Virginia',
    'WI':'Wisconsin',
    'WY':'Wyoming'
}
vax_state['Location'] = vax_state['Location'].map(state_name)

In [64]:
state_mask = pd.read_csv('Data/Cleaned/cleaned_mask.csv')
merged_data = merged_data.merge(vax_state, on='Location').merge(state_mask, on='Location')
merged_data.head()

,Location,Employment_2020,Employment_2021,Employment_2022,Inc_Per_Cap_2020,Inc_Per_Cap_2021,Inc_Per_CAp_2022,Life_Exp_2020,Life_Exp_2019,Life_Exp_2018,Employer_2019,Non-Group_2019,Medicaid_2019,Medicare_2019,Military_2019,Uninsured_2019,Employer_2021,Non-Group_2021,Medicaid_2021,Medicare_2021,Military_2021,Uninsured_2021,Population Density per mi²,2010 Population,2020 Population,flu_vaccination_rate_2019,asthma_prevalence,cardiac_mortality_rate,high_bp_prevalence,copd_prevalence,kidney_disease_prevalence,diabetes_prevalence,Physicians,Physicians Rate,Active MO,Active MO Rate,Active DO,Active DO Rate,Exc_deaths_2017,Exc_deaths_2018,Exc_deaths_2019,Exc_deaths_2020,Exc_deaths_2021,Exc_deaths_2022,Dist_Per_100K_2021,Distributed_Per_100k_65Plus_2021,Admin_Per_100K_2021,Admin_Per_100k_65Plus_2021,Administered_Dose1_Pop_Pct_2021,Administered_Dose1_Recip_65PlusPop_Pct_2021,Series_Complete_Pop_Pct_2021,Series_Complete_65PlusPop_Pct_2021,Additional_Doses_Vax_Pct_2021,Additional_Doses_50Plus_Vax_Pct_2021,Additional_Doses_65Plus_Vax_Pct_2021,Dist_Per_100K_2022,Distributed_Per_100k_65Plus_2022,Admin_Per_100K_2022,Admin_Per_100k_65Plus_2022,Administered_Dose1_Pop_Pct_2022,Administered_Dose1_Recip_65PlusPop_Pct_2022,Series_Complete_Pop_Pct_2022,Series_Complete_65PlusPop_Pct_2022,Additional_Doses_Vax_Pct_2022,Additional_Doses_50Plus_Vax_Pct_2022,Additional_Doses_65Plus_Vax_Pct_2022,Second_Booster_50Plus_Vax_Pct,Second_Booster_65Plus_Vax_Pct,Bivalent_Booster_65Plus_Pop_Pct,Mask_Mandate,Mandatory
0,Alabama,2671005,2769464,2869931,45887,50059,50916,73.2,75.2,75.1,0.472,0.055,0.195,0.160,0.021,0.097,0.466,0.060,0.192,0.162,0.021,0.100,100.65,4779736,5024279,39.5,9.4,289.3,39.4,9.3,3.1,12.2,"10,983",224.0,"10,389",211.9,594,12.1,2649,4062,945,20726,29283,14805,164304,947963,115022,205476,58.6,92.8,47.7,79.7,27.4,38.9,49.0,242643,1399940,141328,259917,64.8,95.0,53.0,85.0,39.5,53.1,62.7,37.9,43.5,20.2,07/16/2020,Yes
1,Alaska,430840,443047,457687,61898,65662,68635,76.6,77.7,78.0,0.484,0.035,0.213,0.100,0.053,0.115,0.433,0.041,0.257,0.107,0.053,0.108,1.3,710231,733391,37.0,9.7,178.5,32.8,4.6,1.8,7.1,"2,101",287.2,"1,825",249.5,276,37.7,417,492,639,1422,3483,1783,180516,1441840,140316,225626,65.1,93.3,56.4,84.2,34.8,52.8,64.1,282456,2256080,178247,307195,72.8,95.0,64.9,89.2,49.0,67.5,78.1,51.8,59.8,37.6,NaN,No
2,Arizona,3920033,4086802,4287595,52133,56420,58442,76.3,78.8,78.7,0.451,0.052,0.210,0.161,0.015,0.111,0.450,0.054,0.213,0.162,0.015,0.106,64.96,6392017,7151502,37.1,9.8,181.8,29.9,6.0,3.6,9.8,"18,343",252.0,"15,989",219.7,"2,353",32.3,3522,4281,1785,30955,40347,20653,173088,962728,144603,236197,67.4,95.0,57.1,83.6,30.4,44.0,51.7,261578,1454920,196497,332661,77.1,95.0,65.8,90.3,49.5,65.0,71.8,51.8,58.9,34.6,NaN,No
3,Arkansas,1639829,1686444,1755536,47147,51636,52618,73.8,75.7,75.6,0.420,0.054,0.262,0.159,0.014,0.091,0.411,0.056,0.270,0.156,0.015,0.092,58.43,2915918,3011524,40.1,9.3,284.3,38.2,9.7,3.7,12.2,"6,500",215.4,"6,132",203.2,368,12.2,3054,2199,1434,11285,15391,8969,165324,952344,125929,211199,62.7,92.6,51.2,78.3,30.2,44.3,55.4,266250,1533720,159188,282457,69.6,95.0,56.7,83.8,44.2,59.8,69.9,46.0,53.0,29.7,07/20/2020,Yes
4,California,23154091,23934549,25300974,70061,76991,77036,79.0,80.9,80.8,0.480,0.066,0.253,0.114,0.009,0.078,0.470,0.070,0.265,0.117,0.008,0.070,258.21,37253956,39538223,40.7,7.8,192.5,26.6,4.2,2.8,9.4,"113,718",287.8,"107,383",271.8,"6,333",16.0,17241,13434,1029,93875,133206,89195,193006,1306260,167055,246904,82.9,95.0,66.2,86.8,33.5,48.6,58.0,293125,1983860,219183,338897,84.3,95.0,74.4,92.7,58.4,73.6,81.2,54.9,63.0,41.8,07/18/2020,Yes


In [65]:
merged_data['Population Density per mi²'] = merged_data['Population Density per mi²'].str.replace(',', '', regex=True).astype(float)
merged_data['Physicians'] = merged_data['Physicians'].str.replace(',', '', regex=True).astype(float)
merged_data['Active DO'] = merged_data['Active DO'].str.replace(',', '', regex=True).astype(float)
merged_data['Active MO'] = merged_data['Active MO'].str.replace(',', '', regex=True).astype(float)
merged_data['Mandatory'] = merged_data['Mandatory'].map({'Yes':1, 'No':0})

In [66]:
merged_data.dtypes

Location                                        object
Employment_2020                                  int64
Employment_2021                                  int64
Employment_2022                                  int64
Inc_Per_Cap_2020                                 int64
Inc_Per_Cap_2021                                 int64
Inc_Per_CAp_2022                                 int64
Life_Exp_2020                                  float64
Life_Exp_2019                                  float64
Life_Exp_2018                                  float64
Employer_2019                                  float64
Non-Group_2019                                 float64
Medicaid_2019                                  float64
Medicare_2019                                  float64
Military_2019                                  float64
Uninsured_2019                                 float64
Employer_2021                                  float64
Non-Group_2021                                 float64
Medicaid_2

In [67]:
merged_data.isnull().sum()

Location                                        0
Employment_2020                                 0
Employment_2021                                 0
Employment_2022                                 0
Inc_Per_Cap_2020                                0
Inc_Per_Cap_2021                                0
Inc_Per_CAp_2022                                0
Life_Exp_2020                                   0
Life_Exp_2019                                   0
Life_Exp_2018                                   0
Employer_2019                                   0
Non-Group_2019                                  0
Medicaid_2019                                   0
Medicare_2019                                   0
Military_2019                                   0
Uninsured_2019                                  0
Employer_2021                                   0
Non-Group_2021                                  0
Medicaid_2021                                   0
Medicare_2021                                   0


In [68]:
merged_data['asthma_prevalence'].fillna(8.4, inplace=True)
merged_data['kidney_disease_prevalence'].fillna(52.1, inplace=True)
merged_data['flu_vaccination_rate_2019'].fillna(38, inplace=True)
merged_data['high_bp_prevalence'].fillna(33, inplace=True)
merged_data['copd_prevalence'].fillna(5.2, inplace =True)
merged_data['diabetes_prevalence'].fillna(9.5, inplace=True)

In [69]:
merged_data.to_csv('Data/Merged_state_data.csv', index=False)

### Modeling 

In [70]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn import metrics

In [71]:
merged_data.dropna(inplace=True)

#### Model 1 - Benchmark

In [97]:
X = merged_data.drop(columns=['Location', 'Mask_Mandate','Exc_deaths_2017', 'Exc_deaths_2018','Exc_deaths_2019',
                              'Exc_deaths_2020', 'Exc_deaths_2021','Exc_deaths_2022'])
y = merged_data[['Exc_deaths_2021']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [100]:
lr1 = LinearRegression()
lr1.fit(X_train, y_train)

LinearRegression()

In [101]:
lr1.score(X_train, y_train)

1.0

In [102]:
lr1.score(X_test, y_test)

-1.9669490347275214

#### Model 2

In [94]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [95]:
lr2 = LinearRegression()
lr2.fit(X_train_sc, y_train)

LinearRegression()

In [96]:
lr2.score(X_train_sc, y_train), lr2.score(X_test_sc, y_test)

(1.0, -0.0810448121786922)

#### Model 3

In [82]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
knn.score(X_train, y_train), knn.score(X_test, y_test)

(0.7474726985948272, 0.7752529507195249)

#### Model 4

In [83]:
knn2 = KNeighborsRegressor()
knn2.fit(X_train_sc, y_train)
knn2.score(X_train_sc, y_train), knn2.score(X_test_sc, y_test)

(0.6255310009795341, 0.459159590923177)

#### Model 5

In [84]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
tree.score(X_train, y_train), tree.score(X_test, y_test)

(1.0, 0.6490272630611302)

#### Model 6

In [85]:
tree2 = DecisionTreeRegressor()
tree2.fit(X_train_sc, y_train)
tree2.score(X_train_sc, y_train), tree2.score(X_test_sc, y_test)

(1.0, 0.7454567967226995)

#### Model 7 

In [86]:
bag1 = BaggingRegressor()
bag1.fit(X_train , y_train)
bag1.score(X_train, y_train), bag1.score(X_test, y_test)

(0.9671084244443612, 0.7756343757942666)

#### Model 8

In [87]:
bag2 = BaggingRegressor()
bag2.fit(X_train_sc, y_train)
bag2.score(X_train_sc, y_train), bag2.score(X_test_sc, y_test)

(0.9412297643730094, 0.6779138482787275)

#### Model 9

In [88]:
rf1 = RandomForestRegressor()
rf1.fit(X_train, y_train)
rf1.score(X_train, y_train), rf1.score(X_test, y_test)

(0.9405926051307385, 0.8261410141189144)

#### Model 10

In [89]:
rf2 = RandomForestRegressor()
rf2.fit(X_train_sc, y_train)
rf2.score(X_train_sc, y_train), rf2.score(X_test_sc,y_test)

(0.9452305167514972, 0.8151190555056762)